In [ ]:
!pip install ktrain
!pip install pydub
import time

In [ ]:
# Install dependencies
!pip install wget pydub wave tqdm vosk
!apt-get ffmpeg

# Download Vosk model
!mkdir models
!wget -P models/ https://alphacephei.com/vosk/models/vosk-model-en-us-0.22.zip
!unzip models/vosk-model-en-us-0.22.zip -d models/ && rm models/vosk-model-en-us-0.22.zip
!wget -P models/ https://alphacephei.com/vosk/models/vosk-model-en-in-0.4.zip
!unzip models/vosk-model-en-in-0.4.zip -d models/ && rm models/vosk-model-in-us-0.4.zip

English model vosk-model-en-us-0.22

Indian English model vosk-model-en-in-0.4

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [ ]:
import ktrain
from ktrain import text
from sklearn.datasets import fetch_20newsgroups

In [ ]:
categories = ['alt.atheism','soc.religion.christian','comp.graphics','sci.med','rec.sport.baseball']

In [ ]:
train = fetch_20newsgroups(
    subset = 'train',
    categories = categories,
    shuffle = True,
    random_state = 0
)

In [ ]:
test = fetch_20newsgroups(
    subset = 'test',
    categories = categories,
    shuffle = True,
    random_state = 0
)

In [ ]:
test.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [ ]:
test.target

array([0, 4, 2, ..., 2, 3, 0])

In [ ]:
test.target_names

['alt.atheism',
 'comp.graphics',
 'rec.sport.baseball',
 'sci.med',
 'soc.religion.christian']

In [ ]:
X_train = train.data
Y_train = train.target

X_test = test.data
Y_test = test.target

In [ ]:
len(X_train),len(Y_train)

(2854, 2854)

In [ ]:
model_name = 'distilbert-base-uncased'

trans = text.Transformer(model_name,maxlen=512,class_names=categories)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
train_data = trans.preprocess_train(X_train,Y_train)
test_data = trans.preprocess_test(X_test,Y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 291
	95percentile : 820
	99percentile : 1757


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 323
	95percentile : 894
	99percentile : 2394


In [ ]:
model = trans.get_classifier()

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

In [ ]:
learner = ktrain.get_learner(model,train_data=train_data,val_data=test_data,batch_size=16)


In [ ]:
#learner.lr_find(show_plot=True,max_epochs=10)

In [ ]:
learner.fit_onecycle(1e-4, 1)



begin training using onecycle policy with max lr of 0.0001...
179/179 [==============================] - 409s 2s/step - loss: 0.1062 - accuracy: 0.9720 - val_loss: 0.1622 - val_accuracy: 0.9521


In [ ]:
learner.validate(class_names=categories)

                        precision    recall  f1-score   support

           alt.atheism       0.88      0.93      0.91       319
soc.religion.christian       0.97      0.96      0.97       389
         comp.graphics       0.98      0.99      0.98       397
               sci.med       0.98      0.93      0.95       396
    rec.sport.baseball       0.94      0.95      0.95       398

              accuracy                           0.95      1899
             macro avg       0.95      0.95      0.95      1899
          weighted avg       0.95      0.95      0.95      1899



array([[297,   1,   1,   4,  16],
       [ 11, 373,   3,   1,   1],
       [  2,   1, 392,   1,   1],
       [  9,   9,   5, 368,   5],
       [ 18,   0,   1,   1, 378]])

In [ ]:
predictor = ktrain.get_predictor(learner.model,preproc=trans)

In [ ]:
from pydub import AudioSegment
import os

def mp3_to_wav(source, skip=0, excerpt=False):

    sound = AudioSegment.from_mp3(source) # load source
    sound = sound.set_channels(1) # mono
    sound = sound.set_frame_rate(16000) # 16000Hz

    if excerpt:
        excrept = sound[skip*1000:skip*1000+30000] # 30 seconds - Does not work anymore when using skip
        output_path = os.path.splitext(source)[0]+"_excerpt.wav"
        excrept.export(output_path, format="wav")
    else:
        audio = sound[skip*1000:]
        output_path = os.path.splitext(source)[0]+".wav"
        audio.export(output_path, format="wav")

    return output_path

In [ ]:
wave_file = mp3_to_wav('cough.wav',0,False)

In [ ]:
from vosk import Model, KaldiRecognizer, SetLogLevel
from tqdm.notebook import tqdm
import wave
import os
import json
outfil=open('Transcription.txt',"w")
outfilee=open('Conf.txt',"w")
def transcript_file(input_file, model_path):
    start = time.time()
    # Check if file exists
    if not os.path.isfile(input_file):
        raise FileNotFoundError(os.path.basename(input_file) + " not found")

    # Check if model path exists
    if not os.path.exists(model_path):
        raise FileNotFoundError(os.path.basename(model_path) + " not found")

    # open audio file
    wf = wave.open(input_file, "rb")

    # check if wave file has the right properties
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getcomptype() != "NONE":
        raise TypeError("Audio file must be WAV format mono PCM.")

    # Initialize model
    model = Model(model_path)
    rec = KaldiRecognizer(model, wf.getframerate())

    # Get file size (to calculate progress bar)
    file_size = os.path.getsize(input_file)

    # Run transcription
    pbar = tqdm(total=file_size)

    # To store our results
    transcription = []
    rec.SetWords(True)

    while True:
        data = wf.readframes(4000) # use buffer of 4000
        pbar.update(len(data))
        if len(data) == 0:
            pbar.set_description("Transcription finished")
            break
        if rec.AcceptWaveform(data):
            # Convert json output to dict

            result_dict = json.loads(rec.Result())
            x=outfilee.write(str(result_dict))
            print(result_dict)
            print("\n")

            # Extract text values and append them to transcription list
            transcription.append(result_dict.get("text", ""))

    # Get final bits of audio and flush the pipeline
    final_result = json.loads(rec.FinalResult())
    x=outfilee.write(str(final_result))
    outfilee.close()
    print(final_result)

    transcription.append(final_result.get("text", ""))

    transcription_text = ' '.join(transcription)
    end = time.time()
    print("Time taken :", end-start)
    return transcription_text

**US English Transcription**

In [ ]:
transcription = transcript_file(wave_file, "/content/models/vosk-model-en-us-0.22")

**Indian** **English** **Transcription**

In [ ]:
transcription = transcript_file(wave_file, "/content/models/vosk-model-en-in-0.4")

In [ ]:
transcription='i have a blurry vision and i can't see in the darkness would is the reason doctor''


'i have some terrible broadlands way and i am dating during the night'

In [ ]:
n=outfil.write(transcription)
outfil.close()

**Old json conversion**

In [ ]:
!pip install -U spacy
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_lg-0.4.0.tar.gz
import scispacy
import spacy
import en_core_sci_lg
from spacy import displacy

**Medical file prediction**

In [ ]:
s= predictor.predict(transcription)
flg=0;
ar=predictor.predict_proba(transcription)
if(s=='sci.med'):
  flg=1
  print(ar[3]*100,"%","Medical file",)

else:
  print(100-ar[3]*100,"%","Non medical file")



In [ ]:
if(flg==1):
  import json
  text = transcription

  nlp = en_core_sci_lg.load()
  doc = nlp(text)

  displacy_image = displacy.render(doc, jupyter=True,style='ent')
  key = list(doc.ents)
  d = {}
  for x in key:
    x=str(x)
    d[x]=text
  y = json.dumps(d)
  print(y)
else:
  print("The given file is not related to medical field")